In [1]:
!python --version

Python 3.9.7


In [34]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager-core).
Your token has been saved to C:\Users\farras\.huggingface\token
Login successful


In [35]:
import pandas as pd
import numpy as np

import re
from sklearn.model_selection import train_test_split

import transformers
import tensorflow as tf

import sentencepiece



In [36]:
pip install rouge_score

Note: you may need to restart the kernel to use updated packages.


In [37]:
from datasets import load_dataset, load_metric

# raw_datasets = load_dataset("xsum")
metric = load_metric("rouge")

In [38]:
df_sum = pd.read_csv(r"C:\SAINS DATA\SEMESTER 5\NLP\data_berita.csv")
df_sum

,teks_berita,text_summary
0,"MEDAN, KOMPAS.com - Sebuah video viral di medi...",baca juga tolak damai suami dari bidan yang se...
1,"JAKARTA, KOMPAS.com - Presiden Joko Widodo had...",presiden joko widodo hadir dalam acara gerakan...
2,"JAKARTA, KOMPAS.com - Jaringan Advokat Tamban...",jaringan advokat tambang jatam mendorong kapol...
3,"YOGYAKARTA, KOMPAS.com - ASEAN Tourism Festiva...",asean tourism festival atf akan digelar pada h...
4,KOMPAS.com - Football vs Soccer menjadi ramai ...,football vs soccer menjadi ramai diperbincangk...
...,...,...
1377,"Kompas.com - 31/Oct/2021, 21:28 WIB Orang past...",baca juga ini makna musim gugur bagi orang jep...
1378,"Kompas.com - 9/Sep/2021, 02:22 WIB OhayoJepang...",pada musim ini tempat tempat yang terkenal unt...
1379,"Kompas.com - 24/Aug/2021, 11:12 WIB OhayoJepan...",aug wibohayojepang sebuah restoran bernama pep...
1380,"Kompas.com - 18/Jul/2022, 08:56 WIB OhayoJepan...",baca juga tempat wisata musim panas menakjubka...


In [39]:
df_sum.duplicated().sum()

113

In [40]:
df_sum = df_sum.drop_duplicates()
df_sum

,teks_berita,text_summary
0,"MEDAN, KOMPAS.com - Sebuah video viral di medi...",baca juga tolak damai suami dari bidan yang se...
1,"JAKARTA, KOMPAS.com - Presiden Joko Widodo had...",presiden joko widodo hadir dalam acara gerakan...
2,"JAKARTA, KOMPAS.com - Jaringan Advokat Tamban...",jaringan advokat tambang jatam mendorong kapol...
3,"YOGYAKARTA, KOMPAS.com - ASEAN Tourism Festiva...",asean tourism festival atf akan digelar pada h...
4,KOMPAS.com - Football vs Soccer menjadi ramai ...,football vs soccer menjadi ramai diperbincangk...
...,...,...
1377,"Kompas.com - 31/Oct/2021, 21:28 WIB Orang past...",baca juga ini makna musim gugur bagi orang jep...
1378,"Kompas.com - 9/Sep/2021, 02:22 WIB OhayoJepang...",pada musim ini tempat tempat yang terkenal unt...
1379,"Kompas.com - 24/Aug/2021, 11:12 WIB OhayoJepan...",aug wibohayojepang sebuah restoran bernama pep...
1380,"Kompas.com - 18/Jul/2022, 08:56 WIB OhayoJepan...",baca juga tempat wisata musim panas menakjubka...


In [41]:
def delete_baca_juga(text):
    text = re.sub(r'baca juga', ' ', text) # Ganti spasi
    return text

In [42]:
df_sum['text_summary'] = df_sum['text_summary'].apply(delete_baca_juga)
df_sum

C:\Users\farras\AppData\Local\Temp/ipykernel_20424/2242207513.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sum['text_summary'] = df_sum['text_summary'].apply(delete_baca_juga)


,teks_berita,text_summary
0,"MEDAN, KOMPAS.com - Sebuah video viral di medi...",tolak damai suami dari bidan yang selingkuh ...
1,"JAKARTA, KOMPAS.com - Presiden Joko Widodo had...",presiden joko widodo hadir dalam acara gerakan...
2,"JAKARTA, KOMPAS.com - Jaringan Advokat Tamban...",jaringan advokat tambang jatam mendorong kapol...
3,"YOGYAKARTA, KOMPAS.com - ASEAN Tourism Festiva...",asean tourism festival atf akan digelar pada h...
4,KOMPAS.com - Football vs Soccer menjadi ramai ...,football vs soccer menjadi ramai diperbincangk...
...,...,...
1377,"Kompas.com - 31/Oct/2021, 21:28 WIB Orang past...",ini makna musim gugur bagi orang jepangberen...
1378,"Kompas.com - 9/Sep/2021, 02:22 WIB OhayoJepang...",pada musim ini tempat tempat yang terkenal unt...
1379,"Kompas.com - 24/Aug/2021, 11:12 WIB OhayoJepan...",aug wibohayojepang sebuah restoran bernama pep...
1380,"Kompas.com - 18/Jul/2022, 08:56 WIB OhayoJepan...",tempat wisata musim panas menakjubkan di hyo...


In [43]:
df_text = pd.read_csv(r"C:\SAINS DATA\SEMESTER 5\NLP\Clean sampe kinclong.csv")
df_text['ID'] = df_text.index
df_text['summary'] = df_sum['text_summary']
df_text = df_text[['ID', 'teks_berita', 'summary']]

df_text.head(3)

,ID,teks_berita,summary
0,0,Sebuah video viral di media sosial yang memper...,tolak damai suami dari bidan yang selingkuh ...
1,1,Presiden Joko Widodo hadir dalam acara Gerakan...,presiden joko widodo hadir dalam acara gerakan...
2,2,Jaringan Advokat Tambang (JATAM) mendorong Ka...,jaringan advokat tambang jatam mendorong kapol...


In [44]:
df_text.isnull().sum()

ID               0
teks_berita      0
summary        111
dtype: int64

In [45]:
df_text = df_text.dropna()

In [46]:
df_text.isnull().sum()

ID             0
teks_berita    0
summary        0
dtype: int64

In [47]:
data_train = df_text.sample(frac = 0.80, random_state=10)
data_val = df_text.drop(data_train.index)

print(len(data_train) + len(data_val))

1158


In [48]:
data_train.to_csv('train.csv', index = False)
data_val.to_csv('val.csv', index = False)

In [49]:
dict_text = df_text.iloc[:, 1:].to_dict('list')
dict_text

{'teks_berita': ['Sebuah video viral di media sosial yang memperlihatkan seorang pelajar berlari dan memanjat bangunan di sebuah SPBU di Jalan Kapten Soemarsono, Deli Serdang, Sumatera Utara, karena dikejar tiga orang yang membawa tongkat dan celurit. Dari video yang diunggah di sejumlah akun Instagram, terlihat dua orang mengejar korban dengan tongkat panjang dan diikuti oleh satu orang yang membawa celurit. Korban mencoba melarikan diri dengan turun dan berlari menjauh. Namun, dia tetap dikejar. Saat itu terlihat banyak kendaraan yang sedang mengisi bahan bakar minyak. Seorang perempuan petugas SPBU berlari ke arah temannya karena ketakutan dengan peristiwa yang terjadi. Korban terlihat berhasil kabur dari tiga orang yang mengejarnya. Namun, terlihat paha kirinya sudah mengeluarkan darah. Di video lainnya yang masih berkaitan dengan peristiwa itu, terlihat siswa tersebut sudah tergeletak mengeluarkan banyak darah di sebuah ruangan di SPBU yang sama. Belakangan diketahui bahwa pelajar

In [84]:
# model_checkpoint = "indolem/indobert-base-uncased"
# model_checkpoint = 'indobenchmark/indobert-base-p2'
model_checkpoint = "google/mt5-small"
# model_checkpoint = "google/mt5-base"
# model_checkpoint = "assamim/mt5-small-indonesian-sum"

In [51]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [85]:
from transformers import BertTokenizer, TFAutoModel, AutoTokenizer, T5Tokenizer

# tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/407 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [86]:
tokenizer("jokowi suka makan bakso")

{'input_ids': [1074, 44606, 8414, 13793, 6789, 263, 268, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [87]:
max_input_length = 5000
max_target_length = 2000


def preprocess_function(document, sum):
    inputs = [doc for doc in document]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    lab = [i for i in sum]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            lab, max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [55]:
type(data_train['teks_berita'])
inputs = [doc for doc in data_train['teks_berita']]
type(inputs)

list

In [88]:
token_train = preprocess_function(document = data_train['teks_berita'], 
                                    sum = data_train['summary'])
# token_train

C:\Users\farras\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3578: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [89]:
token_val = preprocess_function(document = data_val['teks_berita'], 
                                    sum = data_val['summary'])

In [90]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, TFAutoModelForSequenceClassification, TFAutoModel, TFAutoModelWithLMHead

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
# model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint)
# model = TFAutoModel.from_pretrained(model_checkpoint)
# model = TFAutoModelWithLMHead.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/691 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

C:\Users\farras\anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at assamim/mt5-small-indonesian-sum.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [91]:
batch_size = 4
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = f"{model_name}-finetuned-kompas"

In [92]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [61]:
type(dict(token_train))

dict

In [93]:
from datasets import Dataset

ds_train = Dataset.from_dict(dict(token_train))
ds_val = Dataset.from_dict(dict(token_val))

train_dataset = model.prepare_tf_dataset(
    ds_train,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator
)

validation_dataset = model.prepare_tf_dataset(
    ds_val,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator
)

generation_dataset = model.prepare_tf_dataset(
    ds_val,
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [63]:
ds_train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 926
})

In [94]:
train_dataset

<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(4, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(4, None), dtype=tf.int64, name=None), 'decoder_input_ids': TensorSpec(shape=(4, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(4, None), dtype=tf.int64, name=None))>

In [65]:
# from datasets import Dataset

# ds_train = Dataset.from_dict(dict(token_train))
# ds_val = Dataset.from_dict(dict(token_val))

# tf_train = ds_train.to_tf_dataset(
#             columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'],
#             batch_size=24,
#             shuffle=True,
#             collate_fn=data_collator
#             )

# tf_val = ds_val.to_tf_dataset(
#             columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'],
#             batch_size=batch_size,
#             shuffle=False,
#             collate_fn=data_collator
#             )

# tf_gen = ds_val.to_tf_dataset(
#             columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'],
#             batch_size=batch_size,
#             shuffle=False,
#             collate_fn=generation_data_collator
#             )

ValueError: Column token_type_ids not found in dataset!

In [ ]:
tf_train
# tf_val
# tf_gen
# ds_train

In [95]:
from transformers import create_optimizer
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_epochs = 2
num_train_steps = len(train_dataset) * num_train_epochs
model_name = model_checkpoint.split("/")[-1]

optimizer, schedule = create_optimizer(
    init_lr=5.6e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

model.compile(optimizer=optimizer)

# # Train in mixed-precision float16
# tf.keras.mixed_precision.set_global_policy("mixed_float16")

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [96]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir=f"{model_name}-kompas", tokenizer=tokenizer
)

model.fit(
    train_dataset, validation_data=validation_dataset, callbacks=[callback], epochs=2
)

Cloning https://huggingface.co/Farras/mt5-small-indonesian-sum-kompas into local empty directory.


Epoch 1/2


C:\Users\farras\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:436: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 128057344 elements. This may consume a large amount of memory.
  warnings.warn(


KeyboardInterrupt: 

In [80]:
pip install PyYAML

Note: you may need to restart the kernel to use updated packages.


In [82]:
model_yaml = model.to_json()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [70]:
from transformers import pipeline

hub_model_id = "Farras/mt5-small-kompas"
summarizer = pipeline("summarization", model=hub_model_id)

Downloading:   0%|          | 0.00/745 [00:00<?, ?B/s]

C:\Users\farras\anaconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\farras\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

ValueError: Could not load model Farras/mt5-small-kompas with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSeq2SeqLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForSeq2SeqLM'>, <class 'transformers.models.mt5.modeling_mt5.MT5ForConditionalGeneration'>, <class 'transformers.models.mt5.modeling_tf_mt5.TFMT5ForConditionalGeneration'>).

In [ ]:
def print_summary(news):
    news = news
    summary = summarizer(news)
    print(f"'>>> Review: {news}'")
    print(f"\n'>>> Summary: {summary}'")

In [ ]:
# from transformers import AdamWeightDecay

# optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
# model.compile(optimizer=optimizer)

In [ ]:
# import nltk


# def metric_fn(eval_predictions):
#     predictions, labels = eval_predictions
#     decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     for label in labels:
#         label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#     # Rouge expects a newline after each sentence
#     decoded_predictions = [
#         "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_predictions
#     ]
#     decoded_labels = [
#         "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
#     ]
#     result = metric.compute(
#         predictions=decoded_predictions, references=decoded_labels, use_stemmer=True
#     )
#     # Extract a few results
#     result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
#     # Add mean generated length
#     prediction_lens = [
#         np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
#     ]
#     result["gen_len"] = np.mean(prediction_lens)

#     return result

In [ ]:
# from transformers.keras_callbacks import PushToHubCallback, KerasMetricCallback
# from tensorflow.keras.callbacks import TensorBoard

# tensorboard_callback = TensorBoard(log_dir="./summarization_model_save/logs")

# # push_to_hub_callback = PushToHubCallback(
# #     output_dir="./summarization_model_save",
# #     tokenizer=tokenizer,
# #     hub_model_id=push_to_hub_model_id,
# # )

# metric_callback = KerasMetricCallback(
#     metric_fn, eval_dataset=generation_dataset, predict_with_generate=True, use_xla_generation=True
# )

# callbacks = [metric_callback, tensorboard_callback]

# model.fit(
#     train_dataset, validation_data=validation_dataset, epochs=1, callbacks=callbacks
# )